# 라이브러리 설치


In [ ]:
!pip install --upgrade pip setuptools wheel

In [ ]:
pip install --no-build-isolation langsmith

In [ ]:
%pip install --user --quiet  google-cloud-aiplatform \
                             google-cloud-storage \
                             google-cloud-pipeline-components \
                             kfp

# 환경 변수 설정

In [ ]:
PROJECT_ID = "ureca-poc-itcen"
LOCATION = "us-central1"
BUCKET_URI = "gs://ureca_test"
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline/"
DATASET_ID = "test_data"
TABLE_ID = "amazon_review_v1"

SERVICE_ACCOUNT = "vertex-api@ureca-poc-itcen.iam.gserviceaccount.com"

# 라이브러리 불러오기

In [ ]:
# 일반 라이브러리
import json
import logging
import time
import uuid
from typing import NamedTuple, List

# Vertex AI 라이브러리
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1.types.pipeline_state import PipelineState

# Kubeflow Pipelines(KFP) 관련 라이브러리
from kfp import compiler, dsl, client
from kfp import components
from kfp.dsl import (
    Artifact, Metrics, Dataset, Input, Model, Output, component
)
import kfp.v2.dsl as dsl

# 로깅 설정
logger = logging.getLogger("logger")
logging.basicConfig(level=logging.INFO)

# Vertex AI 초기화
vertex_ai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# 사전 빌드된 학습용 컨테이너 이미지 URI
TRAIN_IMAGE = vertex_ai.helpers.get_prebuilt_prediction_container_uri(
    framework="sklearn",
    framework_version="1.3",
    accelerator="cpu"
)

# 컴포넌트 파이프라인

In [ ]:
# 데이터 생성 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=[
        "numpy==1.23.5",
        "pandas",
        "google-cloud-bigquery",
        "joblib",
        "db-dtypes",
    ],
)

def create_dataset(
    data_args: dict,                     # 데이터 파라미터
    data_artifact: Output[Artifact],     # JSON 형태 데이터 아티팩트
    dataset: Output[Dataset],            # CSV 형태 데이터셋
):
    # 라이브러리
    import os
    import json
    import pandas as pd
    from google.cloud import bigquery

    # GCP 설정 값
    PROJECT_ID = "ureca-poc-itcen"
    LOCATION = "us-central1"
    DATASET_ID = "test_data"
    TABLE_ID = "amazon_review_v1"

    # BigQuery 클라이언트 초기화
    client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

    # 쿼리 (데이터 조회)
    query = f"""
    SELECT customer_id, product_id, star_rating
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    """

    # CSV 저장을 위한 폴더 생성
    os.makedirs(dataset.path, exist_ok=True)

    # 쿼리 실행 결과 DataFrame으로 변환 및 CSV 저장
    df = client.query(query).to_dataframe()
    df.to_csv(os.path.join(dataset.path, "raw_data.csv"), index=False)

    # JSON 형식으로 변환
    json_data = df.to_json(orient="records", lines=True)

    # JSON 저장을 위한 폴더 생성
    os.makedirs(data_artifact.path, exist_ok=True)

    # JSON 파일 저장
    with open(os.path.join(data_artifact.path, "data.json"), "w") as f:
        json.dump(json_data, f, indent=4)

# 컴포넌트를 YAML 파일로 컴파일
compiler.Compiler().compile(
    create_dataset,
    "create_dataset.yaml"
)


In [ ]:
# 데이터 전처리 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=["numpy==1.23.5", "pandas", "scikit-learn"],
)

def preprocess_dataset(
    input_dataset: Input[Dataset],            # (입력) 원본 데이터셋
    preprocess_args: dict,                    # (입력) 전처리 관련 하이퍼파라미터
    processed_train: Output[Dataset],         # (출력) 전처리된 학습 데이터
    processed_test: Output[Dataset],          # (출력) 전처리된 테스트 데이터
):
    # 라이브러리
    import os
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import train_test_split

    # preprocess_args 내 테스트셋 비율 불러오기
    test_size = float(preprocess_args['hyper_params']['in_test_size'])

    # 데이터셋 불러오기
    df = pd.read_csv(os.path.join(input_dataset.path, "raw_data.csv"))

    # 사용자 ID와 상품 ID를 숫자 인덱스로 매핑
    user_ids = {user: i for i, user in enumerate(df['customer_id'].unique())}
    item_ids = {item: i for i, item in enumerate(df['product_id'].unique())}
    df['user_idx'] = df['customer_id'].map(user_ids)
    df['item_idx'] = df['product_id'].map(item_ids)

    # 학습 데이터와 테스트 데이터로 분할
    train_df, test_df = train_test_split(df, test_size=test_size, random_state=0)

    # 출력 디렉토리 생성
    os.makedirs(processed_train.path, exist_ok=True)
    os.makedirs(processed_test.path, exist_ok=True)

    # 분할된 데이터를 각각 CSV로 저장
    train_df.to_csv(os.path.join(processed_train.path, "train_data.csv"), index=False)
    test_df.to_csv(os.path.join(processed_test.path, "test_data.csv"), index=False)

# 컴포넌트를 YAML 파일로 컴파일
compiler.Compiler().compile(
    preprocess_dataset,
    "preprocess_dataset.yaml"
)

In [ ]:
# 모델 학습 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=["numpy==1.23.5", "pandas", "joblib"],
)

def train_model(
    train_dataset: Input[Dataset],     # (입력) 학습용 데이터셋 (CSV)
    train_args: dict,                  # (입력) 학습 관련 하이퍼파라미터
    model: Output[Model],              # (출력) 학습된 모델 (joblib 형식)
):
    # 라이브러리
    import numpy as np
    import pandas as pd
    import joblib
    import os
    from collections import defaultdict

    # 파라미터 설정값 불러오기
    n_factors = int(train_args['hyper_params']['n_factors'])
    learning_rate = float(train_args['hyper_params']['learning_rate'])
    reg = float(train_args['hyper_params']['reg'])
    num_epochs = int(train_args['hyper_params']['num_epochs'])

    # 학습 데이터 불러오기
    df = pd.read_csv(os.path.join(train_dataset.path, "train_data.csv"))

    # SVD 학습
    num_users = df["user_idx"].max() + 1
    num_items = df["item_idx"].max() + 1

    P = np.random.normal(0, 0.1, (num_users, n_factors))
    Q = np.random.normal(0, 0.1, (num_items, n_factors))

    item_popularity = defaultdict(int, df["item_idx"].value_counts().to_dict())
    item_histories = df.groupby("user_idx")["item_idx"].apply(set).to_dict()

    for epoch in range(num_epochs):
        total_loss = 0

        for _, row in df.iterrows():
            user_idx = row["user_idx"]
            item_idx = row["item_idx"]
            rating = row["star_rating"]

            pred = np.dot(P[user_idx], Q[item_idx].T)
            error = rating - pred

            P[user_idx] += learning_rate * (error * Q[item_idx] - reg * P[user_idx])
            Q[item_idx] += learning_rate * (error * P[user_idx] - reg * Q[item_idx])

            total_loss += error ** 2

    # 모델 저장을 위한 디렉토리 생성
    os.makedirs(model.path, exist_ok=True)

    # 모델 저장 (행렬 및 부가 정보 포함)
    model_data = {
        "P": P,
        "Q": Q,
        "num_users": num_users,
        "num_items": num_items,
        "item_popularity": dict(item_popularity),
        "item_histories": {k: list(v) for k, v in item_histories.items()}
    }
    joblib.dump(model_data, os.path.join(model.path, "model.joblib"))

# 컴포넌트를 YAML 파일로 컴파일
compiler.Compiler().compile(
    train_model,
    "train_model_svd.yaml"
)


In [ ]:
# 모델 평가 컴포넌트
@component(
    base_image="python:3.10",
    packages_to_install=["numpy==1.23.5", "pandas", "scikit-learn"],
)

def evaluate_model(
    evaluate_args: dict,               # (입력) 평가 관련 하이퍼파라미터
    model: Input[Model],               # (입력) 학습된 모델 (joblib)
    test_data: Input[Dataset],         # (입력) 테스트 데이터셋 (CSV)
    output_artifact: Output[Artifact], # (출력) 평가 결과 (JSON)
    metrics: Output[Metrics],          # (출력) 파이프라인에 기록되는 평가 지표
):
    # 라이브러리
    import os
    import numpy as np
    import pandas as pd
    import joblib
    import json
    from sklearn.metrics import mean_squared_error, mean_absolute_error

    # 테스트 데이터 불러오기
    df = pd.read_csv(os.path.join(test_data.path, "test_data.csv"))

    # 학습된 모델 불러오기
    model_data = joblib.load(os.path.join(model.path, "model.joblib"))
    P, Q = model_data["P"], model_data["Q"]
    num_users, num_items = model_data["num_users"], model_data["num_items"]
    item_popularity = model_data["item_popularity"]
    item_histories = model_data["item_histories"]

    # 예측 및 평가
    predictions = []
    predicted_items = []
    actual_items = []
    errors = []

    for _, row in df.iterrows():
        user_idx = row["user_idx"]
        item_idx = row["item_idx"]

        if user_idx < num_users and item_idx < num_items:
            pred = np.dot(P[user_idx], Q[item_idx].T)
        else:
            pred = np.mean(df["star_rating"])

        predictions.append(pred)
        predicted_items.append(item_idx)
        actual_items.append(item_idx)
        errors.append(row["star_rating"] - pred)

    predictions = np.array(predictions)
    errors = np.array(errors)

    # 평가 지표 계산
    rmse_value = np.sqrt(mean_squared_error(df["star_rating"], predictions))
    total_items = len(item_popularity)
    unique_recommended = len(set(predicted_items))
    coverage = unique_recommended / total_items if total_items > 0 else 0

    novelty_scores = [-np.log2(item_popularity.get(item, 1) + 1) for item in predicted_items]
    novelty = np.mean(novelty_scores) if novelty_scores else 0

    diversity_scores = []
    for i in range(len(predicted_items)):
        for j in range(i + 1, len(predicted_items)):
            if predicted_items[i] in Q and predicted_items[j] in Q:
                sim = np.dot(Q[predicted_items[i]], Q[predicted_items[j]]) / (
                    np.linalg.norm(Q[predicted_items[i]]) * np.linalg.norm(Q[predicted_items[j]])
                )
                diversity_scores.append(1 - sim)
    diversity = np.mean(diversity_scores) if diversity_scores else 0

    serendipity_scores = [
        item not in item_histories.get(user_idx, [])
        for user_idx, item in zip(df["user_idx"], predicted_items)
    ]
    serendipity = np.mean(serendipity_scores) if serendipity_scores else 0

    # 평가지표 저장
    metrics.log_metric("Model", "SVD (Matrix Factorization)")
    metrics.log_metric("RMSE", rmse_value)
    metrics.log_metric("Coverage", coverage)
    metrics.log_metric("Novelty", novelty)
    metrics.log_metric("Diversity", diversity)
    metrics.log_metric("Serendipity", serendipity)

    # 결과를 JSON 파일로 저장
    output_data = {
        "Model": "SVD (Matrix Factorization)",
        "RMSE": rmse_value,
        "Coverage": coverage,
        "Novelty": novelty,
        "Diversity": diversity,
        "Serendipity": serendipity,
    }

    os.makedirs(output_artifact.path, exist_ok=True)
    with open(os.path.join(output_artifact.path, "output.json"), 'w') as f:
        json.dump(output_data, f, indent=4)

# 컴포넌트를 YAML 파일로 컴파일
compiler.Compiler().compile(
    evaluate_model,
    "evaluate_model.yaml"
)


In [ ]:
# 모델 배포 컴포넌트 (학습된 모델을 vertex ai에 업로드 및 서빙용 엔드포인트 생성 )
@dsl.component(
    base_image="python:3.10",
    packages_to_install=[
        'google-cloud-aiplatform',  # Vertex AI SDK
        'pandas',
        'scikit-learn'
    ]
)

def deploy_model(
    deploy_args: dict,                  # (입력) 배포 관련 하이퍼파라미터
    model: Input[Model],                # (입력) 학습된 모델 (joblib)
    vertex_endpoint: Output[Artifact],  # (출력) 생성된 Endpoint 정보
    vertex_model: Output[Model],        # (출력) Vertex AI 모델 정보
):
    # 라이브러리
    from google.cloud import aiplatform

    # 파라미터 설정값 불러오기
    project_id = deploy_args['project_id']
    display_name = deploy_args['display_name']
    container_image_uri = deploy_args['container_image_uri']
    machine_type = deploy_args['machine_type']
    model_name = deploy_args['model_name']

    # Vertex AI 초기화
    aiplatform.init(project=project_id)

    # 모델 업로드 (학습된 모델을 Vertex AI Model Registry에 등록)
    deployed_model = aiplatform.Model.upload(
        display_name=model_name,
        artifact_uri=model.path,
        serving_container_image_uri=container_image_uri
    )

    # Endpoint 생성 및 모델 배포
    endpoint = deployed_model.deploy(machine_type=machine_type)

    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

# 컴포넌트를 YAML 파일로 컴파일
compiler.Compiler().compile(
    deploy_model,
    "deploy_model.yaml"
)


In [ ]:
# 전체 파이프라인 정의 및 YAML 컴파일 함수
def build_pipeline(
    pipeline_name: str,               # 파이프라인 이름
    pipeline_desc: str,               # 파이프라인 설명
    pipeline_root: str,               # 파이프라인 아티팩트가 저장될 GCS 경로
    component_yaml_files: dict        # 컴포넌트별 YAML 파일
):

    # 파이프라인 정의
    @dsl.pipeline(
        name=pipeline_name,
        description=pipeline_desc,
        pipeline_root=pipeline_root,
    )
    def module_pipeline(
        data_args: dict,
        preprocess_args: dict,
        train_args: dict,
        evaluate_args: dict,
        deploy_args: dict
    ):
        # 라이브러리
        from kfp import components

        # 각 컴포넌트 불러오기
        create_dataset_comp = components.load_component_from_file(
            component_yaml_files['create_dataset_file']
        )
        preprocess_dataset_comp = components.load_component_from_file(
            component_yaml_files['preprocess_dataset_file']
        )
        train_model_comp = components.load_component_from_file(
            component_yaml_files['train_model_file']
        )
        evaluate_model_comp = components.load_component_from_file(
            component_yaml_files['evaluate_model_file']
        )
        deploy_model_comp = components.load_component_from_file(
            component_yaml_files['deploy_model_file']
        )

        # 데이터 생성 컴포넌트 실행
        create_dataset_task = create_dataset_comp(data_args=data_args)

        # 데이터 전처리 컴포넌트 실행
        preprocess_dataset_task = preprocess_dataset_comp(
            preprocess_args=preprocess_args,
            input_dataset=create_dataset_task.outputs['dataset']
        )

        # 모델 학습 컴포넌트 실행
        train_model_task = train_model_comp(
            train_args=train_args,
            train_dataset=preprocess_dataset_task.outputs['processed_train']
        )

        # 모델 평가 컴포넌트 실행
        evaluate_model_task = evaluate_model_comp(
            evaluate_args=evaluate_args,
            model=train_model_task.outputs['model'],
            test_data=preprocess_dataset_task.outputs['processed_test']
        )

        # 모델 배포 컴포넌트 실행
        deploy_model_task = deploy_model_comp(
            deploy_args=deploy_args,
            model=train_model_task.outputs['model']
        )

    # 전체 파이프라인을 YAML 파일로 컴파일
    compiler.Compiler().compile(
        pipeline_func=module_pipeline,
        package_path="svd_pipeline.yaml"
    )


In [ ]:
# 각 컴포넌트의 YAML 파일
component_yaml_files = {
    'create_dataset_file': 'create_dataset.yaml',
    'preprocess_dataset_file': 'preprocess_dataset.yaml',
    'train_model_file': 'train_model_svd.yaml',
    'evaluate_model_file': 'evaluate_model.yaml',
    'deploy_model_file': 'deploy_model.yaml',
}

# 전체 파이프라인 빌드 및 YAML 컴파일 실행
build_pipeline(
    pipeline_name="svd_pipeline",                      # 파이프라인 이름
    pipeline_desc="desc for svd_pipeline",             # 파이프라인 설명
    pipeline_root=PIPELINE_ROOT,                       # 파이프라인 결과가 저장될 GCS 경로
    component_yaml_files=component_yaml_files          # 각 컴포넌트 정의 YAML 파일 경로
)


In [ ]:
# Vertex AI 파이프라인 실행
job = aiplatform.PipelineJob(
    display_name="svd_pipeline",
    template_path="svd_pipeline.yaml",

    # 파라미터 입력
    parameter_values={
        'data_args': {},

        'preprocess_args': {
            'hyper_params': {
                'in_test_size': 0.2
            }
        },

        'train_args': {
            'hyper_params': {
                'n_factors': 40,
                'learning_rate': 0.01,
                'reg': 0.05,
                'num_epochs': 50
            }
        },

        'evaluate_args': {},

        'deploy_args': {
            'project_id': 'ureca-poc-itcen',
            'display_name': 'svd',
            'container_image_uri': 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-5:latest',
            'machine_type': 'n1-standard-16',
            'model_name': 'svd'
        },
    },

    pipeline_root=PIPELINE_ROOT,
    enable_caching=True
)

# Vertex AI 파이프라인 실행
job.run(service_account=SERVICE_ACCOUNT)
